+++
title = "Crawling with BeautifulSoup"
id = "Dev"
+++

# Crawling with requests and BeautifulSoup 

> 크롤링 할 때 마다, 자잘한 메서드 이름 까먹어서 이것 저것 이전 작업물들을 열어보게 된다. 한 곳에 모아놓기 1편 BeautifulSoup 

> 개인적으로는 beautifulsoup 으로 할 수 있는 작업 이면, 이것으로 하는 것을 더 좋아한다.(이유는 더 가볍고 **빠르니까!!**)

> 그러나, 정보를 입력 하는 등의 웹 상에서의 행동이 필요하다면 Selenium을 쓴다. 대부분은 bs로 가능하던데 그 때 그 때 판단해서 사용한다. 

> * 이용 추천 - API(있다면, 이걸 쓰는게 가장 굿) -> bs -> Selenium(정말 안되면 사용하라, 브라우져를 직접 씌우니까 무겁다.)

****Flow****

1. Requests로 웹 정보를 받아온다.(html, xml, json $\cdots$)

2. Beautifulsoup으로 html을 파싱한다. 또는 json 파싱을 한다. (xml은 다른 라이브러리를 쓰는게 좋다.[elementtree](https://www.datacamp.com/community/tutorials/python-xml-elementtree))

3. 원하는 정보를 얻는다 ! 

### Requests
- http://docs.python-requests.org/en/master/

--------------------------------
** < practical example > ** 
##### 네이버 주식 데이터 가져오기
- api 사용 : json 파싱을 한다. (*파싱 : Data 형태를 바꿔준다.*)

- 네이버 주식 페이지에서 주식 데이터를 가져와 데이터 프레임으로 만들기
- http://m.stock.naver.com


networks -> XHR -> header -> request URL -> http://m.stock.naver.com/api/mystock/getMyGroupNameList.nhn?1517905359444


--------------------------------
** < practical example > ** 
##### 네이버 주식 데이터 가져오기
- api 사용 : json 파싱을 한다. (*파싱 : Data 형태를 바꿔준다.*)

- 네이버 주식 페이지에서 주식 데이터를 가져와 데이터 프레임으로 만들기
- http://m.stock.naver.com



networks -> XHR -> header -> request URL -> http://m.stock.naver.com/api/mystock/getMyGroupNameList.nhn?1517905359444


In [2]:
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=" + str(pageSize) + "&page=" + str(page)

def get_data(url):
    response = requests.get(url)
    json_info = response.json()
    companys = json_info["result"]["itemList"]
    df = pd.DataFrame(columns=["종목", "시세", "전일비", "등락율", "시가총액", "거래량"])
    for company in companys:
        df.loc[len(df)] = {
            "종목":company["nm"],
            "시세":company["nv"],
            "전일비":company["cv"],
            "등락율":company["cr"],
            "시가총액":company["mks"],
            "거래량":company["aq"],
        }
    return df


url = make_url(100,1)
df = get_data(url)
df.tail()

,종목,시세,전일비,등락율,시가총액,거래량
95,대우조선해양,27300,300,1.11,29267,409962
96,현대엘리베이,107000,-500,-0.47,28862,1246867
97,유한양행,235500,-3000,-1.26,28753,32390
98,한화,38300,-700,-1.79,28709,356358
99,GS리테일,37200,-50,-0.13,28644,158686


In [ ]:
import requests

client_key = 'CsODwdUTyG9vOI1uIeIf'
client_secret = 'YmIx0GW8JG'
# 별도 quote_plus() 메서드등 처리할 필요 없음. requests 객체가 알아서 해줌
naver_url = 'https://openapi.naver.com/v1/search/news.json?query=스마트폰'

header_params = {"X-Naver-Client-Id":client_key, "X-Naver-Client-Secret":client_secret}
# headers= header_params 는 header 변경시에만 필요하고, 그렇지 않으면, requests.get(원하는 URL) 만 해도 됨
response = requests.get(naver_url, headers = header_params)
# 별도 json.loads() 라이브러리 메서드 사용하지 않아도, reqeusts 라이브러리에 있는 json() 메서드로 간단히 처리 가능함
# print(response.json())
# print(response.text)

# HTTP 응답 코드는 status_code 에 저장됨
if(response.status_code == 200):
    data = response.json()
    print(data['items'][0]['title'])
    print(data['items'][0]['description'])
else:
    print("Error Code:" + response.status_code)

### BeautifulSoup

- https://www.crummy.com/software/BeautifulSoup/bs4/doc/

HTML을 가져와서, parsing 을 해주는 역할은 BeautifulSoup이 수행, 보통 Css selector, Xpath를 이용해서 원하는 정보에 접근 한다. 




In [17]:
from bs4 import BeautifulSoup
import re

html = "<html> \
            <body> \
                <h1 id='title'>[1]크롤링이란?</h1> \
                <p class='cssstyle'>웹페이지에서 필요한 데이터를 추출하는 것</p> \
                <p id='body' align='center'>파이썬을 중심으로 다양한 웹크롤링 기술 발달</p> \
            </body> \
        </html>"

soup = BeautifulSoup(html, "html.parser")

# 태그로 검색 방법
title_data = soup.find('h1')

print(title_data)
print(title_data.string)
print(title_data.get_text())

# 가장 먼저 검색되는 태그를 반환
paragraph_data = soup.find('p')

print(paragraph_data)
print(paragraph_data.string)
print(paragraph_data.get_text())


# 태그에 있는 id로 검색 (javascript 예를 상기!)
title_data = soup.find(id='title')

print(title_data)
print(title_data.string)
print(title_data.get_text())


# HTML 태그와 CSS class를 활용해서 필요한 데이터를 추출하는 방법1
paragraph_data = soup.find('p', class_='cssstyle')

print(paragraph_data)
print(paragraph_data.string)
print(paragraph_data.get_text())


# HTML 태그와 CSS class를 활용해서 필요한 데이터를 추출하는 방법2
paragraph_data = soup.find('p', 'cssstyle')

print(paragraph_data)
print(paragraph_data.string)
print(paragraph_data.get_text())


# HTML 태그와 태그에 있는 속성:속성값을 활용해서 필요한 데이터를 추출하는 방법
paragraph_data = soup.find('p', attrs = {'align': 'center'})
print(paragraph_data)
print(paragraph_data.string)
print(paragraph_data.get_text())

# find_all() 관련된 모든 데이터를 리스트 형태로 추출하는 함수
paragraph_data = soup.find_all('p')

print(paragraph_data)
print(paragraph_data[0].get_text())
print(paragraph_data[1].get_text())

# * **string 검색**
#  - 태그가 아닌 문자열 자체로 검색
#  - 문자열, 정규표현식 등등으로 검색 가능
#    - 문자열 검색의 경우 한 태그내의 문자열과 exact matching인 것만 추출
#    - 이것이 의도한 경우가 아니라면 정규표현식 사용

res = requests.get('http://v.media.daum.net/v/20170518153405933')
soup = BeautifulSoup(res.content, 'html5lib')

print (soup.find_all(string='오대석'))
print (soup.find_all(string=['[이주의해시태그-#네이버-클로바]쑥쑥 크는 네이버 AI', '오대석']))
print (soup.find_all(string='AI'))
print (soup.find_all(string=re.compile('AI'))[0])
# print (soup.find_all(string=re.compile('AI')))

<h1 id="title">[1]크롤링이란?</h1>
[1]크롤링이란?
[1]크롤링이란?
<p class="cssstyle">웹페이지에서 필요한 데이터를 추출하는 것</p>
웹페이지에서 필요한 데이터를 추출하는 것
웹페이지에서 필요한 데이터를 추출하는 것
<h1 id="title">[1]크롤링이란?</h1>
[1]크롤링이란?
[1]크롤링이란?
<p class="cssstyle">웹페이지에서 필요한 데이터를 추출하는 것</p>
웹페이지에서 필요한 데이터를 추출하는 것
웹페이지에서 필요한 데이터를 추출하는 것
<p class="cssstyle">웹페이지에서 필요한 데이터를 추출하는 것</p>
웹페이지에서 필요한 데이터를 추출하는 것
웹페이지에서 필요한 데이터를 추출하는 것
<p align="center" id="body">파이썬을 중심으로 다양한 웹크롤링 기술 발달</p>
파이썬을 중심으로 다양한 웹크롤링 기술 발달
파이썬을 중심으로 다양한 웹크롤링 기술 발달
[<p class="cssstyle">웹페이지에서 필요한 데이터를 추출하는 것</p>, <p align="center" id="body">파이썬을 중심으로 다양한 웹크롤링 기술 발달</p>]
웹페이지에서 필요한 데이터를 추출하는 것
파이썬을 중심으로 다양한 웹크롤링 기술 발달
['오대석']
['[이주의해시태그-#네이버-클로바]쑥쑥 크는 네이버 AI', '오대석']
[]
[이주의해시태그-#네이버-클로바]쑥쑥 크는 네이버 AI | Daum 뉴스


## *< practical example >*

In [9]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Naver 실시간 검색어 순위
def naver_top20():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("http://naver.com")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".ah_roll .ah_l .ah_item")
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ah_r").text,
            "keyword":keyword.select_one(".ah_k").text,
        }
    return df
print("< Naver realtime Keywords >")
naver_df = naver_top20()
pp.pprint(naver_df.tail(5))

# 다음 실시간 검색어 순위 
def daum_top10():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("http://daum.net")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select("#mArticle ol.list_hotissue.issue_row.list_mini > li")
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ir_wa").text.replace("위",""),
            "keyword":keyword.select_one(".link_issue").text,
        }
    return df
print("*" * 50)

print("< Daum realtime Keywords >")
daum_df = daum_top10()
pp.pprint(daum_df.tail(5))


# 위의 결과 중에 중복되는 검색어를 출력
result = [keyword for keyword in daum_df["keyword"] if naver_df["keyword"].str.contains(keyword).any() ]
result

< Naver realtime Keywords >
   rank    keyword
15   16      방탄소년단
16   17   해운대 모래축제
17   18  콰이어트 플레이스
18   19        심혜진
19   20         달력
**************************************************
< Daum realtime Keywords >
  rank   keyword
5    6   부처님 오신날
6    7  냉장고를 부탁해
7    8        독전
8    9       홍문종
9   10     성년의 날


['김부겸', '냉장고를 부탁해', '독전']

In [15]:
# File Download

def download(title, download_link):
    response = requests.get(download_link, stream=True)
    download_path ="/Users/MAC/desktop/{}".format(title)
    size = 0
    with open(download_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024): 
            if chunk:
                size += 1024
                f.write(chunk)
    return size


title = "iPhone X is Here — Apple.mp4"
download_link = "http://bit.ly/2FLpRF9"
size = download(title, download_link)
print("download done : {} Mbyte".format(round(size/1024/1024,2)))

download done : 0.0 Mbyte


#### Post 방식

In [ ]:
# 한빛 미디어에 로그인
login_url = 'http://www.hanbit.co.kr/member/login_proc.php'

user = 'input Id'
password = 'and password'

# requests.session 메서드는 해당 reqeusts를 사용하는 동안 cookie를 header에 유지하도록 하여
# 세션이 필요한 HTTP 요청에 사용됩니다.
session = requests.session()

params = dict()
params['m_id'] = user
params['m_passwd'] = password

# javascrit(jQuery) 코드를 분석해보니, 결국 login_proc.php 를 m_id 와 m_passwd 값과 함께
# POST로 호출하기 때문에 다음과 같이 requests.session.post() 메서드를 활용하였습니다.
# 실제코드: <form name="frm"  id="frm"  action="#" method="post">
res = session.post(login_url, data = params) 

# 응답코드가 200 즉, OK가 아닌 경우 에러를 발생시키는 메서드입니다.
res.raise_for_status() 

# 'Set-Cookie'로 PHPSESSID 라는 세션 ID 값이 넘어옴을 알 수 있다.
# print(res.headers)

# cookie로 세션을 로그인 상태를 관리하는 상태를 확인해보기 위한 코드입니다.
# print(session.cookies.get_dict()) 

# 여기서부터는 로그인이 된 세션이 유지됩니다. session 에 header에는 Cookie에 PHPSESSID가 들어갑니다.
mypage_url = 'http://www.hanbit.co.kr/myhanbit/myhanbit.html'
res = session.get(mypage_url)

# 응답코드가 200 즉, OK가 아닌 경우 에러를 발생시키는 메서드입니다.
res.raise_for_status() 

soup = BeautifulSoup(res.text, 'html.parser')

# Chrome 개발자 도구에서 CSS SELECTOR를 통해 간단히 가져온 CSS SELECTOR 표현식을 사용
he_coin = soup.select_one('#container > div > div.sm_mymileage > dl.mileage_section2 > dd > span')

# 다음과 같이 class를 .mileage_section2 로 그리고 그 하부 태그중에 span이 있다는 식으로 표현도 가능함
# he_coin = soup.select_one('.mileage_section2 span')

print ('mileage is', he_coin.get_text())


## BeautifulSoup 으로 했던 것들

-------------------------------------------

- 날씨 크롤링 from (http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
res = requests.get('http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108')

soup = BeautifulSoup(res.content, 'lxml')

data_by_locations = soup.find_all('location')

# cloumns = ['city']
# rows = ['tmef']
# value = 날씨 데이터 
#data_by_locations[0]

df = pd.DataFrame()
#print(data_by_locations)
time_weather = pd.Series()


for location in data_by_locations:
    city = location.find('city') 
    times = location.find_all('tmef')
    values = location.find_all('wf')
    
    city_name = city.get_text()
    for i in range(len(times)):
        weather = values[i].get_text()
        day = times[i].get_text()
        
        time_weather[day] = weather

    df[city_name] = time_weather
#    indexing_time = time
#    df.set_index(time_weather.keys())

df.to_csv('/Users/Mac/Desktop/weather.csv')
    
df

<div class="alert alert-block alert-warning">
<font color="blue" size="3em"><b>네이버 영화 댓글 크롤링 프로젝트</b></font>
http://movie.naver.com/

<font size="3em"><b>요구사항:</b></font><br>
http://movie.naver.com/movie/running/current.nhn 페이지(현재 상영영화)에서 <br>
<br>
(1) 예매순 1위 ~ 5위에 해당하는 영화 각각의<br>
(2) 개봉후 평점글(140자 평)을 <br>
(3) 호감순 100개 읽어서 출력하기<br>
<br>
<font size="3em"><b>출력 포멧:</b></font><br>
[영화 이름] (호감순 140자 평)<br>
1]호감순 140자 평1<br>
2]호감순 140자 평1<br>
<b>.</b><br>
<b>.</b><br>
<b>.</b><br>
100]호감순 140자 평1<br>
<br>
<font size="3em"><b>고려 사항:</b></font><br>
140자 평이 100개 이하일 경우에는 140자 평 갯수만큼 출력할 것<br>

In [ ]:
import requests
from bs4 import BeautifulSoup

res = requests.get('http://movie.naver.com/movie/running/current.nhn')
soup = BeautifulSoup(res.content, 'html.parser')

movie_list = {}
movies = soup.select("#content > div.article > div > div.lst_wrap > ul > li")

for n in range(5):
    movie = movies[n].find_all("a")
    movie_list[movie[1].string] = "http://movie.naver.com"+movie[0].get('href').replace("basic", "pointWriteFormList")+"&type=after&page="
        
for movie_name, movie_href in movie_list.items():
    print('[{}] (호감순 140자 평)'.format(movie_name))
    comments = []
    i=0
    loop = True
    while loop and i < 10:
        i+=1
        try:
            res = requests.get(movie_href+str(i))
            soup = BeautifulSoup(res.content, 'html.parser')
            comment_list = soup.select("body > div > div > div.score_result > ul")
            for comment in comment_list[0].find_all("p"):
                comments.append(comment.text.replace('BEST','').replace('관람객',''))
        except:
            loop = False
        
    for i, comment in enumerate(comments):
        print('{}] {}'.format(i+1, comment))

#### 네이버 경제 기사 크롤링 

In [ ]:
from bs4 import BeautifulSoup as bs
import requests 
import re 
import time
import datetime

# Get the news for an year
for i in range(365):
    today = datetime.date.today()
    today = today + datetime.timedelta(-i)
    today = ("").join(str(today).split('-'))
    print(today)
    naver_news_today= "http://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&sectionId=101&date={}".format(today)
    print(naver_news_today)
    res = requests.get(naver_news_today)
    soup = bs(res.content, 'html.parser')
    try:
    # get rinks
        rinks = []
        # top3
        top3 = soup.find('div', class_="ranking_top3")
        top3 = top3.select("a[href]")
        for top in top3:
            rinks.append(re.search('/main/\w+/\w+[.a-zA-Z?=&0-9;_]+', top['href']).group())
        rinks = rinks[::2]
        # 4 to 30th ranked news
        for rank in range(4,31):
            all_= soup.find('li', class_="gnum{}".format(rank))
            all_ = all_.select("a[href]")
            for al in all_:
                rinks.append(re.search('/main/\w+/\w+[.a-zA-Z?=&0-9_]+', al['href']).group())
        
        # gathering the day's news
        articles = '{}'.format(today)
        for rink in rinks:
            res = requests.get(base_url+rink)
            soup = bs(res.content, 'html.parser')
            article = re.sub(r'[\n]', '', soup.find("div", id="articleBodyContents").text)
            article = re.sub(r'[\w0-9a-zA-Z_.-]@[\w0-9a-zA-Z_.-]', '', article)
            article = re.sub(r'\w+\.\w+.\w+', '', article)
            article = re.sub(r'무단전재 및 재배포 금지', '', article)
            article = re.sub(r'▶.*', '', article )
            articles += article 
        
        # make txt files
        with open("article{}.txt".format(i), "w") as f:
            f.write(articles)
        
    except : 
        print("{} is missed".format(i))
